# Prompt Tuning
apply prompt tuning to  model of choice using [Parameter-Efficient Fine-Tuning (PEFT) library developed by HuggingFace](https://huggingface.co/docs/peft/index). This PEFT library supports multiple methods to reduce the number of parameters for fine-tuning, including prompt tuning and LoRA.



1. Apply prompt tuning to model of choice
1. Fine-tune on provided dataset
1. Save and share model to HuggingFace hub
1. Conduct inference using the fine-tuned model
1. Compare outputs from randomly- and text-initialized fine-tuned model vs. foundation model

In [ ]:
%pip install peft==0.4.0

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.1 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "bigscience/bloomz-560m"

tokenizer = AutoTokenizer.from_pretrained(model_name)
foundation_model = AutoModelForCausalLM.from_pretrained(model_name)

Before doing any fine-tuning, we will ask the model to generate a new phrase to the following input sentence.

In [ ]:
input1 = tokenizer("Two things are infinite: ", return_tensors="pt")
foundation_outputs = foundation_model.generate(
    input_ids=input1["input_ids"],
    attention_mask=input1["attention_mask"],
    max_new_tokens=20,
    eos_token_id=tokenizer.eos_token_id
    )
print(tokenizer.batch_decode(foundation_outputs, skip_special_tokens=True))

['Two things are infinite:  the number of people and the number of things']


The output is not too bad. However, the dataset BLOOMZ is pre-trained on doesn't cover anything about inspirational English quotes. Therefore, we are going to fine-tune `bloomz-560m` on [a dataset called `Abirate/english_quotes`](https://huggingface.co/datasets/Abirate/english_quotes)  containing exclusively inspirational English quotes, with the hopes of using the fine-tuned version to generate more quotes later!

In [ ]:
from datasets import load_dataset

data = load_dataset("Abirate/english_quotes", cache_dir=DA.paths.datasets+"/datasets")

data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)
train_sample = data["train"].select(range(50))
display(train_sample)

/databricks/python_shell/dbruntime/huggingface_patches/datasets.py:13: UserWarning: During large dataset downloads, there could be multiple progress bar widgets that can cause performance issues for your notebook or browser. To avoid these issues, use `datasets.utils.logging.disable_progress_bar()` to turn off the progress bars.
  warnings.warn(


Found cached dataset json (/dbfs/mnt/dbacademy-datasets/llm-foundation-models/v01-raw/datasets/Abirate___json/Abirate--english_quotes-6e72855d06356857/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /dbfs/mnt/dbacademy-datasets/llm-foundation-models/v01-raw/datasets/Abirate___json/Abirate--english_quotes-6e72855d06356857/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-768c685e1cb83483.arrow


Dataset({
    features: ['quote', 'author', 'tags', 'input_ids', 'attention_mask'],
    num_rows: 50
})

In [ ]:
from peft import  get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit

peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.RANDOM,
    num_virtual_tokens=4,
    tokenizer_name_or_path=model_name
)
peft_model = get_peft_model(foundation_model, peft_config)
print(peft_model.print_trainable_parameters())

trainable params: 4,096 || all params: 559,218,688 || trainable%: 0.0007324504863471229
None


In [ ]:
from transformers import TrainingArguments
import os

output_directory = os.path.join(DA.paths.working_dir, "peft_outputs")

if not os.path.exists(DA.paths.working_dir):
    os.mkdir(DA.paths.working_dir)
if not os.path.exists(output_directory):
    os.mkdir(output_directory)

training_args = TrainingArguments(
    output_dir=output_directory, # Where the model predictions and checkpoints will be written
    no_cuda=True, # This is necessary for CPU clusters.
    auto_find_batch_size=True, # Find a suitable batch size that will fit into memory automatically
    learning_rate= 3e-2, # Higher learning rate than full fine-tuning
    num_train_epochs=1 # Number of passes to go through the entire fine-tuning dataset
)

In [ ]:
from transformers import Trainer, DataCollatorForLanguageModeling

trainer = Trainer(
    model=peft_model, # We pass in the PEFT version of the foundation model, bloomz-560M
    args=training_args,
    train_dataset=train_sample,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False) # mlm=False indicates not to use masked language modeling
)

trainer.train()

/databricks/python/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


TrainOutput(global_step=7, training_loss=3.6653319767543246, metrics={'train_runtime': 143.4858, 'train_samples_per_second': 0.348, 'train_steps_per_second': 0.049, 'total_flos': 12033285439488.0, 'train_loss': 3.6653319767543246, 'epoch': 1.0})

## Save model

In [ ]:
import time

time_now = time.time()
peft_model_path = os.path.join(output_directory, f"peft_model_{time_now}")
trainer.model.save_pretrained(peft_model_path)

## Inference

You can load the model from the path that you have saved to before, and ask the model to generate text based on our input before!

In [ ]:
from peft import PeftModel

loaded_model = PeftModel.from_pretrained(foundation_model,
                                         peft_model_path,
                                         is_trainable=False)

In [ ]:
loaded_model_outputs = loaded_model.generate(
    input_ids=input1["input_ids"],
    attention_mask=input1["attention_mask"],
    max_new_tokens=7,
    eos_token_id=tokenizer.eos_token_id
    )
print(tokenizer.batch_decode(loaded_model_outputs, skip_special_tokens=True))

['Two things are infinite:  the number of people and the number']


## Text initialization

Our fine-tuned, randomly initialized model did pretty well on the quote above. Let's now compare it with the text initialization method.

Notice that all we are changing is the `prompt_tuning_init` setting and we are also providing a concise text prompt.

API docs
* [prompt_tuning_init_text](https://huggingface.co/docs/peft/main/en/package_reference/tuners#peft.PromptTuningConfig.prompt_tuning_init_text)

In [ ]:
text_peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    prompt_tuning_init_text="Generate inspirational quotes", # this provides a starter for the model to start searching for the best embeddings
    num_virtual_tokens=3, # this doesn't have to match the length of the text above
    tokenizer_name_or_path=model_name
)
text_peft_model = get_peft_model(foundation_model, text_peft_config)
print(text_peft_model.print_trainable_parameters())

trainable params: 3,072 || all params: 559,217,664 || trainable%: 0.0005493388706691496
None


In [ ]:
text_trainer = Trainer(
    model=text_peft_model,
    args=training_args,
    train_dataset=train_sample,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

text_trainer.train()

/databricks/python/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=7, training_loss=3.153122220720564, metrics={'train_runtime': 134.5159, 'train_samples_per_second': 0.372, 'train_steps_per_second': 0.052, 'total_flos': 12033285439488.0, 'train_loss': 3.153122220720564, 'epoch': 1.0})

In [ ]:
# Save the model
time_now = time.time()
text_peft_model_path = os.path.join(output_directory, f"text_peft_model_{time_now}")
text_trainer.model.save_pretrained(text_peft_model_path)

# Load model
loaded_text_model = PeftModel.from_pretrained(
    foundation_model,
    text_peft_model_path,
    is_trainable=False
)

# Generate output
text_outputs = text_peft_model.generate(
    input_ids=input1["input_ids"],
    attention_mask=input1["attention_mask"],
    max_new_tokens=7,
    eos_token_id=tokenizer.eos_token_id
)

print(tokenizer.batch_decode(text_outputs, skip_special_tokens=True))

['Two things are infinite:  the number of people who will hear']


You can see that text initialization doesn't necessarily perform better than random initialization.

In [ ]:
from huggingface_hub import login

os.environ["huggingface_key"] = dbutils.secrets.get("llm_scope", "huggingface_key")
hf_token = os.environ["huggingface_key"]
login(token=hf_token)

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
File <command-1738672828157799>:3
      1 from huggingface_hub import login
----> 3 os.environ["huggingface_key"] = dbutils.secrets.get("llm_scope", "huggingface_key")
      4 hf_token = os.environ["huggingface_key"]
      5 login(token=hf_token)

File /databricks/python_shell/dbruntime/dbutils.py:240, in DBUtils.SecretsHandler.get(self, scope, key)
    239 def get(self, scope, key):
--> 240     return self.entry_point.getDbutils().preview().secret().get(scope, key)

File /databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py:1322, in JavaMember.__call__(self, *args)
   1316 command = proto.CALL_COMMAND_NAME +\
   1317     self.command_header +\
   1318     args_command +\
   1319     proto.END_COMMAND_PART
   1321 answer = self.gateway_client.send_command(command)
-> 1322 return_value = get_return_value(
   1323     an

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:

hf_username = <FILL_IN_WITH_YOUR_HUGGINGFACE_USERNAME>
peft_model_id = f"{hf_username}/bloom_prompt_tuning_{time_now}"
trainer.model.push_to_hub(peft_model_id, use_auth_token=True)

### Inference from model in HuggingFace hub

In [ ]:
from peft import PeftModel, PeftConfig

config = PeftConfig.from_pretrained(peft_model_id)
foundation_model = AutoModelForCausalLM.from_pretrained(peft_config.base_model_name_or_path)
peft_random_model = PeftModel.from_pretrained(foundation_model, peft_model_id)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1738672828157804>:3
      1 from peft import PeftModel, PeftConfig
----> 3 config = PeftConfig.from_pretrained(peft_model_id)
      4 foundation_model = AutoModelForCausalLM.from_pretrained(peft_config.base_model_name_or_path)
      5 peft_random_model = PeftModel.from_pretrained(foundation_model, peft_model_id)

NameError: name 'peft_model_id' is not defined

In [ ]:
online_model_outputs = peft_random_model.generate(
    input_ids=input1["input_ids"],
    attention_mask=input1["attention_mask"],
    max_new_tokens=7,
    eos_token_id=tokenizer.eos_token_id
    )

print(tokenizer.batch_decode(online_model_outputs, skip_special_tokens=True))